## Multi-Class Classification

In [ ]:
import pandas as pd
import numpy as np
import requests
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

class CropRecommendationSystem:
    """
    Complete Crop Recommendation System with ML and Weather Integration
    """
    
    def __init__(self, csv_path=None):
        self.model = None
        self.scaler = None
        self.trained = False
        
        if csv_path:
            self.load_and_train(csv_path)
    
    def load_and_train(self, csv_path):
        """Load data and train the model"""
        print("Loading dataset...")
        df = pd.read_csv(csv_path)
        
        # Prepare data
        X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
        y = df['label']
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Scale features
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        print("Training model...")
        self.model = RandomForestClassifier(
            n_estimators=200,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )
        self.model.fit(X_train_scaled, y_train)
        
        # Evaluate
        y_pred = self.model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"Model trained successfully!")
        print(f"Accuracy: {accuracy * 100:.2f}%")
        
        self.trained = True
    
    def save_model(self, model_path='crop_model.pkl', scaler_path='scaler.pkl'):
        """Save trained model and scaler"""
        if not self.trained:
            print("Model not trained yet!")
            return
        
        joblib.dump(self.model, model_path)
        joblib.dump(self.scaler, scaler_path)
        print(f"Model saved to {model_path}")
        print(f"Scaler saved to {scaler_path}")
    
    def load_model(self, model_path='crop_model.pkl', scaler_path='scaler.pkl'):
        """Load pre-trained model and scaler"""
        self.model = joblib.load(model_path)
        self.scaler = joblib.load(scaler_path)
        self.trained = True
        print("Model loaded successfully!")
    
    def get_weather(self, api_key, city):
        """Fetch weather data from WeatherAPI"""
        base_url = "https://api.weatherapi.com/v1/current.json"
        params = {'key': api_key, 'q': city, 'lang': 'english'}
        
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            
            return {
                'temperature': data['current']['temp_c'],
                'humidity': data['current']['humidity'],
                'rainfall': data['current']['precip_mm'],
                'city': data['location']['name']
            }
        except:
            return None
    
    def predict(self, N, P, K, temperature, humidity, ph, rainfall, top_n=5):
        """Make crop prediction"""
        if not self.trained:
            print("Model not trained yet!")
            return None
        
        # Prepare input
        input_data = np.array([[N, P, K, temperature, humidity, ph, rainfall]])
        input_scaled = self.scaler.transform(input_data)
        
        # Predict
        prediction = self.model.predict(input_scaled)[0]
        probabilities = self.model.predict_proba(input_scaled)[0]
        
        # Get top N recommendations
        crop_names = self.model.classes_
        results = pd.DataFrame({
            'Crop': crop_names,
            'Suitability_Score': probabilities * 100
        }).sort_values('Suitability_Score', ascending=False).head(top_n)
        
        return {
            'best_crop': prediction,
            'recommendations': results.to_dict('records'),
            'input_conditions': {
                'N': N, 'P': P, 'K': K,
                'temperature': temperature,
                'humidity': humidity,
                'ph': ph,
                'rainfall': rainfall
            }
        }
    
    def predict_with_weather(self, N, P, K, ph, city, weather_api_key, top_n=5):
        """Predict crop using real-time weather data"""
        weather = self.get_weather(weather_api_key, city)
        
        if not weather:
            print("Could not fetch weather. Using default values.")
            weather = {
                'temperature': 25, 
                'humidity': 70, 
                'rainfall': 100,
                'city': city
            }
        
        result = self.predict(
            N, P, K,
            weather['temperature'],
            weather['humidity'],
            ph,
            weather['rainfall'],
            top_n
        )
        
        result['weather'] = weather
        return result
    
    def display_results(self, result):
        """Display prediction results in a formatted way"""
        print("\n" + "="*60)
        print("CROP RECOMMENDATION SYSTEM - RESULTS")
        print("="*60)
        
        if 'weather' in result:
            print(f"\nLocation: {result['weather']['city']}")
            print(f"\nCurrent Weather:")
            print(f"  Temperature: {result['weather']['temperature']}°C")
            print(f"  Humidity: {result['weather']['humidity']}%")
            print(f"  Rainfall: {result['weather']['rainfall']}mm")
        
        print(f"\nSoil Conditions:")
        ic = result['input_conditions']
        print(f"  Nitrogen (N): {ic['N']} kg/ha")
        print(f"  Phosphorus (P): {ic['P']} kg/ha")
        print(f"  Potassium (K): {ic['K']} kg/ha")
        print(f"  pH Level: {ic['ph']}")
        print(f"  Temperature: {ic['temperature']}°C")
        print(f"  Humidity: {ic['humidity']}%")
        print(f"  Rainfall: {ic['rainfall']}mm")
        
        print(f"\n{'='*60}")
        print(f"BEST RECOMMENDED CROP: {result['best_crop'].upper()}")
        print(f"{'='*60}")
        
        print("\nTop Recommendations:")
        print("-" * 60)
        for i, rec in enumerate(result['recommendations'], 1):
            print(f"{i}. {rec['Crop']:15s} - Suitability: {rec['Suitability_Score']:.2f}%")
        print("="*60)

# ============= USAGE EXAMPLE =============

# Initialize system and train
system = CropRecommendationSystem('Crop_recommendation.csv')

# Save the model
system.save_model()

# Example 1: Predict with manual input
result = system.predict(
    N=90, P=42, K=43,
    temperature=21.0,
    humidity=82.0,
    ph=6.5,
    rainfall=200.0,
    top_n=5
)
system.display_results(result)

# Example 2: Predict with real-time weather
result = system.predict_with_weather(
    N=85, P=58, K=41, ph=7.0,
    city="Egypt",
    weather_api_key="e35a98d4e3324f499ee205025251311",
    top_n=5
)
system.display_results(result)